---
title: "Topics in Econometrics and Data Science: Tutorial 6"

---

#### General Note

You will very likely find the solution to these exercises online. We, however, strongly encourage you to work on these exercises without doing so. Understanding someone else’s solution is very different from coming up with your own. Use the lecture notes and try to solve the exercises independently.

# Section 1 cont'd: Basic Statistics

## Exercise 3: Hypothesis Testing (II)
Now we want to test whether the proportion (probability of success) in a group equals a certain given value.

1. Use `help(stats.binomtest)` to make yourself familiar with the [`binomtest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binomtest.html) function.

In [1]:
import scipy.stats as stats
#help(stats.binomtest)

2. A poll taken in $2003$ among $200$ Europeans found that only $16\%$ favored the policies of the United States. Perform a test of significance to see whether this is significantly different from the $50\%$ proportion of Americans in favor of these policies.

In [2]:
stats.binomtest(32,200,0.5) # two-sided test

BinomTestResult(k=32, n=200, alternative='two-sided', statistic=0.16, pvalue=1.818080038171794e-23)

3. A new drug therapy is tested. Of $50$ patients in the study, $40$ had no recurrence in their illness after $18$ months. With no drug therapy, the expected percentage of no recurrence would have been $75\%$. Does the data support the hypothesis that this percentage has increased? What is the $p$-value? Why is useful to consider a one-sided test?

In [3]:
stats.binomtest(40,50,0.75) # two-sided test

BinomTestResult(k=40, n=50, alternative='two-sided', statistic=0.8, pvalue=0.514121165666749)

In [4]:
stats.binomtest(40,50,0.75, alternative = "greater") # One-sided hypothesis

# We want to know whether the drug therapy increases the proportion of patients with no recurrence. 
# A two-sided test would also consider the possibility of a decrease, which is not relevant here. 
# A one-sided test is more powerful for detecting an increase in the percentage.

BinomTestResult(k=40, n=50, alternative='greater', statistic=0.8, pvalue=0.2622023101895092)

## Exercise 4: Bootstrap
Load the [`lightbulb_lifetime.csv`](https://alexandragibbon.github.io/StatProg-HHU/data/lightbulb_lifetime.csv) data, which contains a (simulated) sample of lifespans of lightbulbs measured in hours. 

In [5]:
import pandas as pd
import numpy as np
import os
path = "C:/Users/DICE/Dropbox/Statistical Programming/StatProg-HHU-oct24"
os.chdir(path)
lightbulb_lifetime = pd.read_csv('data/lightbulb_lifetime.csv', sep=';', na_values=".")
lightbulb_lifetime.head()

,0
0,572.691989
1,26.268241
2,797.757928
3,571.500317
4,545.361518


1. Estimate the median lifetime of a lightbulb.\
**Hint:** you can use `.values` to convert pandas dataframes into arrays and `.flatten()` to create vectors from arrays.

In [9]:
lightbulb_lifetime = lightbulb_lifetime.values
lightbulb_lifetime = lightbulb_lifetime.flatten()
med = np.median(lightbulb_lifetime)
print(med)

596.2420166828964


2. The sample median is an asymptotically normal distributed estimator for the median. Try to estimate the variance of the sample median with bootstrap.  
**Hint**: Use `np.random.choice()`, additionally `np.empty()` could be helpful.

In [12]:
import numpy as np
import math
import scipy.stats as stats

np.random.seed(0)

n = 200
B = 500
alpha = 0.05
quantile = stats.norm.ppf(q = 1-alpha/2)
bootstrap_sample1 = np.empty((B, n))
for j in range(B):
     bootstrap_sample1[j] = np.random.choice(lightbulb_lifetime,n)
#or even better
bootstrap_sample = np.random.choice(lightbulb_lifetime,(B,n))
bootstrap_est = np.median(bootstrap_sample,axis =1)
#print(bootstrap_est)

In [14]:
var_est = np.var(bootstrap_est)
print(var_est)
# using the asymptotic normal distribution
confidence_interval1 = (med - np.sqrt(var_est)*quantile,med + np.sqrt(var_est)*quantile)
# using the sample quantiles for the estimator
confidence_interval2 = np.quantile(bootstrap_est, (alpha/2,1-alpha/2))
print(confidence_interval1)
print(confidence_interval2)

2800.3657745531796
(492.5236971543423, 699.9603362114506)
[538.31503576 706.09709512]


3. Show with a simulation that your confidence interval is able to asymptotically satisfy the confidence level. Use standard normal distributed and exponential ( with parameter 0.01) distributed random variables in your simulation.  
**Hint**:`np.random.normal()` and `np.random.exponential()`

#### Here's the second variant (working on pandas DataFrames): 

In [36]:
import pandas as pd
import numpy as np
lightbulb_lifetime = pd.read_csv('data/lightbulb_lifetime.csv', sep=';', na_values=".")
lightbulb_lifetime.head()

,0
0,572.691989
1,26.268241
2,797.757928
3,571.500317
4,545.361518


In [37]:
med = np.median(lightbulb_lifetime)
print(med)

596.2420166828964


In [ ]:
import numpy as np
import math
import scipy.stats as stats
import random

np.random.seed(0)
n = 200
B = 500
alpha = 0.05
quantile = stats.norm.ppf(q = 1-alpha/2)

bootstrap_est = np.empty(B)

for j in range(B):
    indx = np.random.randint(n, size=n)
    bootstrap_est[j] = np.median(lightbulb_lifetime.iloc[indx])

In [39]:
var_est = np.var(bootstrap_est)
# using the asymptotic normal distribution
confidence_interval1 = (med - np.sqrt(var_est)*quantile,med + np.sqrt(var_est)*quantile)

# using the sample quantiles for the estimator
confidence_interval2 = np.quantile(bootstrap_est, (alpha/2, 1-alpha/2))

print(confidence_interval1)
print(confidence_interval2)

(492.71134162812444, 699.7726917376684)
[537.36395253 703.68082448]


Constructing a one-sided confidence interval (here only calculating the lower bound, since the upper bound is $\infty$):

In [40]:
quantile2  = stats.norm.ppf(q = 1-alpha)
# using the asymptotic normal distribution
lower_bound1 = med - np.sqrt(var_est)*quantile2
# using the sample quantiles for the estimator
lower_bound2 = np.percentile(bootstrap_est, (5))
print(lower_bound1)
print(lower_bound2)

509.3563352582254
545.3342596272213


In [18]:
np.random.seed(0)
n = 500
B = 200
l = 10000

In [19]:
mu = 0
sigma = 1
#the true median is zero

alpha = 0.05
quantile = stats.norm.ppf(q = 1-alpha/2)
count = 0
for i in range(1, l):
    sample = np.random.normal(mu, sigma, n)
    med_est = np.median(sample)
    bootstrap_sample = np.random.choice(sample,(B,n))
    bootstrap_est = np.median(bootstrap_sample,axis =1)
    var_est = np.var(bootstrap_est)
    confidence_interval1 = (med_est - np.sqrt(var_est)*quantile,med_est + np.sqrt(var_est)*quantile)
    if confidence_interval1[0] > 0 or confidence_interval1[1] < 0:
        count += 1
print("percentage of rejected null hypothesis:", count/l*100)

percentage of rejected null hypothesis: 6.22


In [20]:
lam = 100
med = np.log(2)*100 #true median

alpha = 0.05
quantile = stats.norm.ppf(q = 1-alpha/2)
count = 0
for i in range(1, l):
    sample = np.random.exponential(lam, n)    
    med_est = np.median(sample)
    bootstrap_sample = np.random.choice(sample,(B,n))
    bootstrap_est = np.median(bootstrap_sample,axis =1)
    var_est = np.var(bootstrap_est)
    confidence_interval1 = (med_est - np.sqrt(var_est)*quantile,med_est + np.sqrt(var_est)*quantile)
    if confidence_interval1[0] > med or confidence_interval1[1] < med:
        count += 1
print("percentage of rejected null hypothesis:", count/l*100)

percentage of rejected null hypothesis: 6.0600000000000005
